In [ ]:
# Individualized Model: Ensemble -- NN + HMM 

In [ ]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import np_utils

In [ ]:
# Data Path: FS4 NEW folder

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p8_data_FS4new.csv'
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

### Train Test Split (for chosen Feature Set)

#### TODO: Specify which train-test split you want

1. Random Shuffle Split: 70-30, 50-50, 30-70, 10-90
2. Chronological Split: 70-30, 50-50, 30-70
3. Session Split: 70-30, 50-50, 30-70, 10-90

In [ ]:
# Session Split: Train on Percentage of Data From Each Session, Sorted Chronologically 

def session_split(data, size):
    sessions = data['session_num'].unique()
    
    first = True
        
    for s in sessions:
        curr = data.loc[data['session_num'] == s]
        curr = curr.sort_values(['timestamp'], ascending=[True])

        y = curr['engagement']
        X = curr.drop(columns=['timestamp', 'session_num', 'engagement'])

        if first:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, shuffle=True)
            first = False
            
        else:
            CX_train, CX_test, Cy_train, Cy_test = train_test_split(X, y, test_size=size, shuffle=True)
                        
            X_train = X_train.append(CX_train)
            X_test = X_test.append(CX_test)
            y_train = y_train.append(Cy_train)
            y_test = y_test.append(Cy_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Sort data (don't need for Random Shuffle, but ok to leave it)

data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [ ]:
# TODO: Uncomment train-test split to run below 

y = data['engagement']
X = data.drop(columns=['timestamp', 'engagement'])

# Choose one of these 


# 1. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

### (Extra)
# Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Random Shuffle Split 10-90
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
### 

#############################

# 2. Chronological Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# 3. Chronological Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# 4. Chronological Split 30-70
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

############################# 

# 5. Session Split 70-30
# X_train, X_test, y_train, y_test = session_split(data, 0.3)

# 6. Session Split 50-50
# X_train, X_test, y_train, y_test = session_split(data, 0.5)

# 7. Session Split 30-70
# X_train, X_test, y_train, y_test = session_split(data, 0.7)

# 8. Session Split 10-90
# X_train, X_test, y_train, y_test = session_split(data, 0.9)

In [ ]:
# Save Session Nums in Order for HMM 

X_train_sess = X_train['session_num']
X_train = X_train.drop(columns=['session_num'])

X_test_sess = X_test['session_num']
X_test = X_test.drop(columns=['session_num'])

X_train_hmm = X_train.copy()
y_train_hmm = y_train.copy()

In [ ]:
# Sort X,y Train dataframes together after split 

X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

### Neural Network Model: Ensemble of 10

In [ ]:
# NN Model (really just a simple classifier): 29 => 1

def get_nn_model():
    num_epochs = 5
    size_batch = 96   

    nn_model = Sequential()

    nn_model.add(Dense(units=1, input_dim=(X_train.shape[1])))
    nn_model.add(Activation('sigmoid'))

    nn_model.summary()

    nn_model.compile(loss='binary_crossentropy',  
                  optimizer='Adam',
                  metrics=['accuracy'])

    nn_model.fit(X_train, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1)
    
    return nn_model

In [ ]:
m1 = get_nn_model()
m2 = get_nn_model()
m3 = get_nn_model()
m4 = get_nn_model()
m5 = get_nn_model()
m6 = get_nn_model()
m7 = get_nn_model()
m8 = get_nn_model()
m9 = get_nn_model()
m10 = get_nn_model()

In [ ]:
def nn_pred(nn, X):
    nn_pred = nn.predict(X)
    nn_pred2 = []
    for i in range(0, len(nn_pred)):
        nn_pred2.append(nn_pred[i][0])
    return nn_pred2

In [ ]:
# Add Predictions from 10 NN's to Original Dataframe 

X_train_nn = X_train.copy()
X_train_nn['m1'] = nn_pred(m1, X_train)
X_train_nn['m2'] = nn_pred(m2, X_train)
X_train_nn['m3'] = nn_pred(m3, X_train)
X_train_nn['m4'] = nn_pred(m4, X_train)
X_train_nn['m5'] = nn_pred(m5, X_train)
X_train_nn['m6'] = nn_pred(m6, X_train)
X_train_nn['m7'] = nn_pred(m7, X_train)
X_train_nn['m8'] = nn_pred(m8, X_train)
X_train_nn['m9'] = nn_pred(m9, X_train)
X_train_nn['m10'] = nn_pred(m10, X_train)

In [ ]:
# Ensemble Model: repredict using 10 NN models 

num_epochs = 5
size_batch = 96   

nn_model = Sequential()

nn_model.add(Dense(units=1, input_dim=(X_train_nn.shape[1])))
nn_model.add(Activation('sigmoid'))

nn_model.summary()

nn_model.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

nn_model.fit(X_train_nn, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1)

#### NN Model Evaluation

In [ ]:
# New test data with predictions from 10 NN models

X_test_nn = X_test.copy()
X_test_nn['m1'] = nn_pred(m1, X_test)
X_test_nn['m2'] = nn_pred(m2, X_test)
X_test_nn['m3'] = nn_pred(m3, X_test)
X_test_nn['m4'] = nn_pred(m4, X_test)
X_test_nn['m5'] = nn_pred(m5, X_test)
X_test_nn['m6'] = nn_pred(m6, X_test)
X_test_nn['m7'] = nn_pred(m7, X_test)
X_test_nn['m8'] = nn_pred(m8, X_test)
X_test_nn['m9'] = nn_pred(m9, X_test)
X_test_nn['m10'] = nn_pred(m10, X_test)

In [ ]:
loss_and_metrics = nn_model.evaluate(X_test_nn, y_test)
print(loss_and_metrics)

In [ ]:
scores = nn_model.predict_proba(X_test_nn)
pred = nn_model.predict_classes(X_test_nn)

In [ ]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

### HMM Model

In [ ]:
# HMM trains on sequences, which is each session. Save Lengths 

train_lengths = X_train_sess.value_counts(sort=False).tolist()
test_lengths = X_test_sess.value_counts(sort=False).tolist()

In [ ]:
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
import random

# HMM tends to get stuck in local minimum. For best results, run several times and pick best

best_model = GaussianHMM(n_components=2, n_iter=5, algorithm = 'viterbi').fit(X_train_hmm, train_lengths)
best_pred = best_model.predict(X_train_hmm, train_lengths)
best = accuracy_score(y_train_hmm, best_pred)
num = 0

for i in range(0,10):
    new_model = GaussianHMM(n_components=2, n_iter=5, algorithm = 'viterbi', init_params="mcs")
    
    # randomly initialize probability matrix 
    a = random.uniform(0, 1)
    b = random.uniform(0, 1)
    new_model.transmat_ = np.array([[a, 1-a], [b, 1-b]])

    new_model = new_model.fit(X_train_hmm, train_lengths)
    
    # evaluate this model and see if better than best so far 
    new_pred = new_model.predict(X_train_hmm, train_lengths)
    new_acc = accuracy_score(y_train_hmm, new_pred)
    print(i, new_acc)
    
    if (new_acc > best):
        best = new_acc
        best_model = new_model
        num = i
        
        
hmm_model = best_model

#### HMM Model Evaluation

In [ ]:
hmm_pred = hmm_model.predict(X_test, test_lengths)
scores = hmm_model.predict_proba(X_test.values, test_lengths)
scores = scores[: ,1]

print("Accuracy for HMM:",accuracy_score(y_test, hmm_pred))
print("AUC:",roc_auc_score(y_test, scores)) 
print(metrics.confusion_matrix(y_test, hmm_pred))
print(metrics.classification_report(y_test, hmm_pred))

### Ensemble Model: NN + HMM 

In [ ]:
def nn_pred(nn, X):
    nn_pred = nn.predict(X)
    nn_pred2 = []
    for i in range(0, len(nn_pred)):
        nn_pred2.append(nn_pred[i][0])
    return nn_pred2

def hmm_pred(X, lengths):
    scores = hmm_model.predict_proba(X.values, lengths)
    scores = scores[: ,1]
    return scores

In [ ]:
# Prep X_train_hmm for X_train_ens:
X_train_ens_nn = X_train_hmm.copy()
X_train_ens_nn['m1'] = nn_pred(m1, X_train_hmm)
X_train_ens_nn['m2'] = nn_pred(m2, X_train_hmm)
X_train_ens_nn['m3'] = nn_pred(m3, X_train_hmm)
X_train_ens_nn['m4'] = nn_pred(m4, X_train_hmm)
X_train_ens_nn['m5'] = nn_pred(m5, X_train_hmm)
X_train_ens_nn['m6'] = nn_pred(m6, X_train_hmm)
X_train_ens_nn['m7'] = nn_pred(m7, X_train_hmm)
X_train_ens_nn['m8'] = nn_pred(m8, X_train_hmm)
X_train_ens_nn['m9'] = nn_pred(m9, X_train_hmm)
X_train_ens_nn['m10'] = nn_pred(m10, X_train_hmm)

In [ ]:
# ENSEMBLE Data

X_train_ens = X_train_hmm.copy()
X_train_ens['nn_model'] = nn_pred(nn_model, X_train_ens_nn)
X_train_ens['hmm_model'] = hmm_pred(X_train_hmm, train_lengths)

y_train_ens = y_train_hmm.copy()

X_test_ens = X_test.copy()
X_test_ens['nn_model'] = nn_pred(nn_model, X_test_nn)
X_test_ens['hmm_model'] = hmm_pred(X_test, test_lengths)

In [ ]:
### NN Ensemble 

In [ ]:
# Shuffle Train Dataset

X_train_ens, bogus_test, y_train_ens, bogus_test = train_test_split(X_train_ens, y_train_ens, test_size=0.0, shuffle=True)
X_train_ens, bogus_test, y_train_ens, bogus_test = train_test_split(X_train_ens, y_train_ens, test_size=0.0, shuffle=True)
X_train_ens, bogus_test, y_train_ens, bogus_test = train_test_split(X_train_ens, y_train_ens, test_size=0.0, shuffle=True)
X_train_ens, bogus_test, y_train_ens, bogus_test = train_test_split(X_train_ens, y_train_ens, test_size=0.0, shuffle=True)
X_train_ens, bogus_test, y_train_ens, bogus_test = train_test_split(X_train_ens, y_train_ens, test_size=0.0, shuffle=True)

In [ ]:
# Ensemble Model (really just a simple classifier): 29 => 1

num_epochs = 5
size_batch = 96   

ens_model = Sequential()

ens_model.add(Dense(units=1, input_dim=(X_train_ens.shape[1])))
ens_model.add(Activation('sigmoid'))

ens_model.summary()

ens_model.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

ens_model.fit(X_train_ens, y_train_ens, epochs=num_epochs, batch_size=size_batch, verbose=1)

#### NN Ensemble Model Evaluation

In [ ]:
loss_and_metrics = ens_model.evaluate(X_test_ens, y_test)
print(loss_and_metrics)

In [ ]:
ens_scores = ens_model.predict_proba(X_test_ens)
ens_pred = ens_model.predict_classes(X_test_ens)

In [ ]:
try:
    print("AUC:",roc_auc_score(y_test, ens_scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, ens_pred))
print(metrics.confusion_matrix(y_test, ens_pred))
print(metrics.classification_report(y_test, ens_pred))

In [ ]:
a = X_test_ens['nn_model']
b = X_test_ens['hmm_model']
c = y_test

In [ ]:
m1_c = 0
m2_c = 0
b_c = 0
n_c = 0

for i in range(77980, 181956):
    if (a[i] == c[i]) and (b[i] == c[i]):
        b_c += 1
    elif (a[i] == c[i]):
        m1_c += 1
    elif (b[i] == c[i]):
        m2_c += 1
    else:
        n_c += 1

In [ ]:
print(m1_c, m2_c, b_c, n_c)

In [ ]:
(m1_c+b_c)/(n_c+b_c+m1_c+m2_c)